In [1]:
#Script to load data
#Code for improved fMRI-bias learning
#OV Jan 2021
####IMPORTS
import tensorflow as tf
import numpy as np
import keras
import freesurfer as fs
from keras import backend as K
from keras import layers as KL
from nibabel import freesurfer as nbfs
#from freesurfer import deeplearn as fsd
#OV: This imports all the local settings, e.g. subjects etc. from the header.py and netparms.py file, make your changes in there as needed
from netparms import *
from header_better import *
import glob
####END IMPORTS
tf.__version__

'2.4.1'

In [2]:
#Surfaces, CNN in- and outputs (functional, angio data, thickness)
surfs_sphere   = []
surfs_pial     = []
surfs_inflated = []
labels_cortex  = []
maps_func      = []
maps_func_mean = []
maps_angles    = []
maps_thickness = []
maps_angio     = []
NumFuncs       = np.zeros(10)

for subj, subject in enumerate(subjects):

   #Read in surfaces and sphere for paramterisation
   #t1 base directory with surfaces	
   anat_dir       = os.path.join(sdir, subject, '%s_T1_base' %(subject))
   
   #Initialise lists for functional and venographgy runs over hemispheres
   labels    = []
   funcs     = []
   func_means= []
   angles    = []
   spheres   = []
   pials     = []
   inflated  = []
   thickness = []
   angios    = []

   #Counter for how many functional data sets we have for a particular subject
   numfunc = 0 

   for hno, hemi in enumerate(hemis):   
   #sphere coordinate system, lh and rh
      fname          = os.path.join(anat_dir,'surf', '%s.sphere.reg' %(hemi))
      sphere_surf    = fs.Surface.read(fname)
      spheres.append(sphere_surf)    #append each hemisphere
      #Pial surface
      fname          = os.path.join(anat_dir,'surf', '%s.pial' % (hemi))
      pial           = fs.Surface.read(fname)
      pials.append(pial)        #append each hemisphere
      #Inflafted surface
      fname          = os.path.join(anat_dir,'surf', '%s.inflated' % (hemi))
      infl           = fs.Surface.read(fname)
      inflated.append(infl)        #append each hemisphere
      #Thickness 
      fname          = os.path.join(anat_dir,'surf', '%s.thickness' %(hemi))
      ov             = fs.Overlay.read(fname)
      thick          = sphere_surf.parameterize(ov)
      thickness.append(thick)  #append each hemisphere)
      #Cortex label, use nibabel to read in, fs is deprecated
      fname          = os.path.join(anat_dir,'label', '%s.cortex.label.mgz' %(hemi))
      ov             = fs.Overlay.read(fname)
      label          = sphere_surf.parameterize(ov)
      labels.append(label)  #append each hemisphere)
      #Angiographic projections
      fname          = os.path.join(anat_dir,'mri', '%s.%s_AngioVeno_projfrac0.0.mgz' %(hemi,subject))
      ov             = fs.Overlay.read(fname)
      angio          = sphere_surf.parameterize(ov)
      angios.append(angio)  #append each hemisphere)
      #Mean of coefficient of variation
      fname          = os.path.join(sdir, subject, '%s.%s_func_CoeffVar_projfrac1.0_mean.mgz' %(hemi,subject))
      ov             = fs.Overlay.read(fname)
      func_mean      = sphere_surf.parameterize(ov)
      func_means.append(func_mean)  #append each hemisphere)

      #Initialise list of functional and orientation data that loops over runs
      func  = []
      angle = []
    
      ### Read in functional data ####
      for func_run in Func_runs:
         dir_name    = os.path.join(sdir, subject, func_run)
        
         #Not all func files could be registered, so had to throw some out
         fname    = os.path.join(dir_name,'%s.%s_%s_filtered_func_CoeffVar2white_projfrac0.6.mgz' %(hemi,subject,func_run))
         if os.path.exists(fname):
            print('Reading subject %s, functional run %s on hemisphere %s ' %(subject, func_run, hemi))
            ov       = fs.Overlay.read(fname)
            #2D representation of the cortex (surface parametrization where axes are 
            #longitude and colatitude), this python code generates parametrization 
            #from the spherical coordinates
            mrisp               = sphere_surf.parameterize(ov) #standardize_image(sphere_surf.parameterize(ov))       
#            test0               = mrisp
#            test00              = ov.data     
            target_shape_no_pad = mrisp.shape
            #data_wrapped        = np.pad(mrisp.data, ((pad,pad),(0,0)), 'wrap')
            #Append all surfaces, i.e. lh, rh, lr, rh,... don't make separate array dimension for hemispheres
            func.append(mrisp)
            #Read in orientation overlays
            fname          = os.path.join(dir_name,'%s.pial2%s_angles.mgz' %(hemi,func_run))
            ov             = fs.Overlay.read(fname)
            tmp            = ov.data[0:,4].copy()
            tmp[tmp>90]    = tmp[tmp>90]-180
            tmp            = np.absolute(tmp)
            mrisp          = sphere_surf.parameterize(tmp)
            angle.append(mrisp)
            #counter
            numfunc        = numfunc + 1
         else:
            pass
       #end functional data
    
      funcs.append(func) #append each hemisphere
      angles.append(angle)
    
   #end hemispheres
   
   surfs_sphere.append(spheres)
   surfs_pial.append(pials)
   surfs_inflated.append(inflated)
   labels_cortex.append(labels)
   maps_func.append(funcs) 	#append each subject [subj, hemis, func_runs]
   #tmp = np.asarray(funcs) #Calculate mean fluctuation map
   #tmp = np.mean(tmp,axis=1)
   maps_func_mean.append(func_means)
   print(np.asarray(maps_func_mean).shape)
   maps_angles.append(angles) #append each subject    [subj, hemis, veno_runs]
   maps_thickness.append(thickness)#append each subject    [subj, hemis, thickness]
   maps_angio.append(angios)
   NumFuncs[subj] = numfunc

#end subjects

np.save('data_train/surfs_sphere.npy',surfs_sphere)
np.save('data_train/surfs_pials.npy',surfs_pial)
np.save('data_train/surfs_inflated.npy',surfs_inflated)
np.save('data_train/labels_cortex.npy',labels_cortex)
np.save('data_train/maps_func.npy',maps_func)
np.save('data_train/maps_func_mean.npy',maps_func_mean)
np.save('data_train/maps_angles.npy',maps_angles)
np.save('data_train/maps_thickness.npy',maps_thickness)
np.save('data_train/maps_angio.npy',maps_angio)
np.save('data_train/NumFuncs.npy',NumFuncs)

Reading subject sub-MSC01, functional run ses-func01 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func02 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func03 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func04 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func05 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func06 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func07 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func08 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func09 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func10 on hemisphere lh 
Reading subject sub-MSC01, functional run ses-func01 on hemisphere rh 
Reading subject sub-MSC01, functional run ses-func02 on hemisphere rh 
Reading subject sub-MSC01, functional run ses-func03 on hemisphere rh 
Reading subject sub-MSC01, functional run ses-func04 on hemisphere rh 
Readin

Reading subject sub-MSC07, functional run ses-func05 on hemisphere lh 
Reading subject sub-MSC07, functional run ses-func06 on hemisphere lh 
Reading subject sub-MSC07, functional run ses-func08 on hemisphere lh 
Reading subject sub-MSC07, functional run ses-func09 on hemisphere lh 
Reading subject sub-MSC07, functional run ses-func10 on hemisphere lh 
Reading subject sub-MSC07, functional run ses-func01 on hemisphere rh 
Reading subject sub-MSC07, functional run ses-func02 on hemisphere rh 
Reading subject sub-MSC07, functional run ses-func03 on hemisphere rh 
Reading subject sub-MSC07, functional run ses-func04 on hemisphere rh 
Reading subject sub-MSC07, functional run ses-func05 on hemisphere rh 
Reading subject sub-MSC07, functional run ses-func06 on hemisphere rh 
Reading subject sub-MSC07, functional run ses-func08 on hemisphere rh 
Reading subject sub-MSC07, functional run ses-func09 on hemisphere rh 
Reading subject sub-MSC07, functional run ses-func10 on hemisphere rh 
(7, 2,

/cluster/visuo/users/olivia/software/python/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
